In [23]:
import json
import os

import requests
from dotenv import load_dotenv
from openai import OpenAI

import openai as client

# open ai assistant 생성
# assistant = client.beta.assistants.create(
#     name="Book Assistant",
#     instructions="You help users with their questions on the files they upload",
#     model="gpt-4-turbo",
#     tools=[{"type": "file_search"}]
# )

assistant_id = 'asst_vSYfxk144i8kWlJbqn7Cb22i'

message_file = client.files.create(
    file=open("../files/sample.txt", "rb"), purpose="assistants"
)

thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know where does Winston live and how his apartment looks like.",
            # "content": "I want you to help me with this file",
            "attachments": [
                {"file_id": message_file.id, "tools": [{"type": "file_search"}]}
            ],
        }
    ]
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
# print("\n".join(citations))

Winston lives in Victory Mansions, which are located in London, specifically in the province of Oceania[0]. His apartment is described as having a hallway that smells of boiled cabbage and old rag mats, suggesting a dreary and dilapidated setting. Inside, the flat is equipped with a device called a telescreen that cannot be completely shut off, which broadcasts a voice reading out figures related to pig-iron production. The flat itself appears quite functional and stark, with Winston noting such details as the frailty and meagerness emphasized by his uniform[0].
